<a href="https://colab.research.google.com/github/agnivachakraborty/LendingClubCaseStudy/blob/master/LendingClubCaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lending Club Case Study**



### Loading Data


In [5]:
## Establish the connection between google drive and colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
## Importing the Libraries required for the analysis
import numpy as np
import pandas as pd

## Read CSV data from the google drive path
df = pd.read_csv('/content/drive/MyDrive/Upgrad/LendingClubCaseStudy/loan.csv',low_memory=False)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


### Data Cleaning

Columns

In [29]:
## Delete columns where all elements are NaN . Have used inplace=True as we do not need these columns for any analysis at all.
# Finding all columns which has all NaN
colsNan = df.isnull().sum() == len(df)
colsNan[colsNan==True]
# Total 54 columns identified with all NaNs
colsNan[colsNan==True].sum()
# Deleting those columns
df.dropna(axis=1, how='all', inplace=True)
df.columns

## Delete columns where all elements are zeroes
zeroes = (df == 0)
# Find columns with only zeroes
cols = zeroes.all()[zeroes.all()].index.to_list()
df = df.drop(cols, axis=1)
df.columns

## Deleting columns which has less than equal to 3 unique values . They don't hep with the analysis.
# Find the count of unique values columns
uniquecount = df.nunique()
# Find columns having count less than 3
todelcols = uniquecount[uniquecount<=3].index.to_list()
todelcols
df = df.drop(todelcols,axis=1)
df.columns

## Deleting columns having all unique values , keeping 'id' column as the Key
# Find columns having count same as total rows
todelcols = uniquecount[uniquecount==len(df)].index.to_list()
todelcols
# Keeping 'id' we will delete the other 2 columns
df = df.drop(columns=['member_id','url'])
df.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'issue_d', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d'],
      dtype='object')

Rows